<a href="https://colab.research.google.com/github/ArghyaPal/ItsArghya/blob/master/MNIST_Joint_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np

In [0]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 300
batch_size = 20


In [0]:
sample_dir = 'samples'
save_dir = 'save'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [0]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5),   # 3 for RGB channels
                                     std=(0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.D_image = nn.Sequential(
                              nn.Linear(image_size, hidden_size),
                              nn.LeakyReLU(0.2))
        self.D_label = nn.Sequential(
                              nn.Linear(10, hidden_size),
                              nn.LeakyReLU(0.2))
        self.D_common = nn.Sequential(
                              nn.Linear(hidden_size*2, hidden_size),
                              nn.LeakyReLU(0.2),
                              nn.Linear(hidden_size, 1),
                              nn.Sigmoid())
        
    def forward(self, image, label):
        image = self.D_image(image)
        label = self.D_label(label)
        common = torch.cat([image, label], 1)
        result = self.D_common(common)
        return result

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.G_common = nn.Sequential(
                              nn.Linear(latent_size, hidden_size),
                              nn.ReLU(),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU())
        self.G_image = nn.Sequential(
                              nn.Linear(hidden_size, image_size),
                              nn.Tanh())
        self.G_label = nn.Sequential(
                              nn.Linear(hidden_size, 10),
                              nn.Softmax(dim=1))
                              #nn.Sigmoid())

    def forward(self, input):
        common = self.G_common(input)
        image = self.G_image(common)
        label = self.G_label(common)
        return image, label

In [0]:
# Device setting
G = Generator().cuda()
D = Discriminator().cuda()

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [0]:
# Statistics to be saved
d_losses = np.zeros(num_epochs)
g_losses = np.zeros(num_epochs)
real_scores = np.zeros(num_epochs)
fake_scores = np.zeros(num_epochs)

In [11]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, original_labels) in enumerate(data_loader):
        
        images = images.view(batch_size, -1).cuda()
        images = Variable(images)
        original_labels = torch.nn.functional.one_hot(original_labels, num_classes=10)
        original_labels = original_labels.view(batch_size, -1).cuda()
        original_labels = Variable(original_labels).float()
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).cuda()
        real_labels = Variable(real_labels)
        fake_labels = torch.zeros(batch_size, 1).cuda()
        fake_labels = Variable(fake_labels)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images, original_labels)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).cuda()
        z = Variable(z)
        fake_images, gen_labels = G(z)
        outputs = D(fake_images, gen_labels)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        # If D is trained so well, then don't update
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).cuda()
        z = Variable(z)
        fake_images, gen_labels = G(z)
        outputs = D(fake_images, gen_labels)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        # if G is trained so well, then don't update
        reset_grad()
        g_loss.backward()
        g_optimizer.step()


         #=================================================================== #
        #                          Update Statistics                          #
        # =================================================================== #
        d_losses[epoch] = d_losses[epoch]*(i/(i+1.)) + d_loss.data*(1./(i+1.))
        g_losses[epoch] = g_losses[epoch]*(i/(i+1.)) + g_loss.data*(1./(i+1.))
        real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().data*(1./(i+1.))
        fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().data*(1./(i+1.))


        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.data, g_loss.data, 
                          real_score.mean().data, fake_score.mean().data))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    print(gen_labels)
    save_image(denorm(fake_images.data), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
    # Save and plot Statistics
    np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
    np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
    np.save(os.path.join(save_dir, 'fake_scores.npy'), fake_scores)
    np.save(os.path.join(save_dir, 'real_scores.npy'), real_scores)
    
    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    plt.plot(range(1, num_epochs + 1), d_losses, label='d loss')
    plt.plot(range(1, num_epochs + 1), g_losses, label='g loss')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'loss.pdf'))
    plt.close()

    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    pylab.ylim(0, 1)
    plt.plot(range(1, num_epochs + 1), fake_scores, label='fake score')
    plt.plot(range(1, num_epochs + 1), real_scores, label='real score')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'accuracy.pdf'))
    plt.close()

    # Save model at checkpoints
    if (epoch+1) % 50 == 0:
        torch.save(G.state_dict(), os.path.join(save_dir, 'G--{}.ckpt'.format(epoch+1)))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D--{}.ckpt'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/300], Step [200/3000], d_loss: 0.3901, g_loss: 1.6256, D(x): 0.85, D(G(z)): 0.14
Epoch [0/300], Step [400/3000], d_loss: 0.5618, g_loss: 2.3561, D(x): 0.90, D(G(z)): 0.26
Epoch [0/300], Step [600/3000], d_loss: 0.4784, g_loss: 2.5864, D(x): 0.81, D(G(z)): 0.04
Epoch [0/300], Step [800/3000], d_loss: 0.1478, g_loss: 4.7134, D(x): 0.98, D(G(z)): 0.10
Epoch [0/300], Step [1000/3000], d_loss: 0.7779, g_loss: 1.8080, D(x): 0.92, D(G(z)): 0.34
Epoch [0/300], Step [1200/3000], d_loss: 0.2596, g_loss: 2.8081, D(x): 0.88, D(G(z)): 0.08


KeyboardInterrupt: ignored